In [19]:
import pandas as pd
import numpy as nps
import plotly.express as px
import chardet

# Need to guess encoding of the file as the default encoding did not work
with open("air-bnb-listings_USA.csv", "rb") as f:
    raw_data = f.read(1000000)

guess = chardet.detect(raw_data)
print(guess)

df = pd.read_csv('air-bnb-listings_USA.csv', encoding=guess['encoding'])

#df.head()

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [20]:
# Cleaning the data

# Drop columns that are not needed
columns_to_drop = ['Name', 'Minimum nights', 'Number of reviews', 'Date last review', 'Number of reviews per month', 'Availibility', 'Updated Date', 'Coordinates']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Drop rows with missing values
df = df.dropna()

# Drop rows with price less than 10
df = df[df['Room Price'] >= 10]


In [ ]:
host_location_counts = (
    df.groupby('Host ID')['Location']
    .nunique()
    .reset_index(name='DistinctLocation')
)

# Count total listings per host
host_listing_counts = (
    df.groupby('Host ID')['Room ID']
    .count()
    .reset_index(name='TotalListings')
)

# Merge these two dataframes
host_info = pd.merge(
    host_location_counts,
    host_listing_counts,
    on='Host ID',
    how='left'
)

# Define "professional host", my definition is someone who has more than one listing in more than one location
host_info['IsProfessional'] = (
    (host_info['TotalListings'] > 1) &
    (host_info['DistinctLocation'] > 1)
)

print("Total Hosts:", host_info.shape[0])

num_professional = host_info['IsProfessional'].sum()
print("Professional Hosts:", num_professional)

# Merge 'IsProfessional' back into the main df
df = pd.merge(
    df,
    host_info[['Host ID','IsProfessional']],
    on='Host ID',
    how='left'
)

# Quick check
df.head(10)

Total Hosts: 139321
Professional Hosts: 6848


,Room ID,Host ID,Neighbourhood,Room type,Room Price,Rooms rent by the host,City,Country,Location,IsProfessional
0,575758,2832150,Five Points,Entire home/apt,200,1,Denver,United states,"United states, Denver, Five Points",False
1,1041934,5811115,Berkeley,Entire home/apt,215,7,Denver,United states,"United states, Denver, Berkeley",False
2,1311993,6658113,CBD,Entire home/apt,90,22,Denver,United states,"United states, Denver, CBD",False
3,1557739,8289288,Cole,Private room,60,7,Denver,United states,"United states, Denver, Cole",True
4,3338717,16853725,Rosedale,Entire home/apt,90,1,Denver,United states,"United states, Denver, Rosedale",False
5,3404692,9599078,Whittier,Entire home/apt,140,1,Denver,United states,"United states, Denver, Whittier",False
6,5814645,17085160,Speer,Entire home/apt,199,1,Denver,United states,"United states, Denver, Speer",False
7,8026291,4657393,University,Private room,69,3,Denver,United states,"United states, Denver, University",False
8,8884899,46489460,Capitol Hill,Entire home/apt,92,1,Denver,United states,"United states, Denver, Capitol Hill",False
9,9296383,47832519,Berkeley,Private room,75,2,Denver,United states,"United states, Denver, Berkeley",False


In [ ]:
# 1) Calculate how many unique professional hosts in each location
location_pro_hosts = (
    df[df['IsProfessional']]
    .groupby('Location')['Host ID']
    .nunique()
    .reset_index(name='NumProfessionalHosts')
    .sort_values('NumProfessionalHosts', ascending=False)
)

top10_pro = location_pro_hosts.head(10)



fig1 = px.bar(
    top10_pro,
    x='NumProfessionalHosts',
    y='Location',
    orientation='h',
    title='(1) Top 10 Locations by Count of Professional Hosts',
    color='NumProfessionalHosts',
    color_continuous_scale='Viridis',
    text='NumProfessionalHosts' 
)


fig1.update_layout(
    height=700,
    margin=dict(l=200, r=50, t=80, b=50),
    xaxis_title='Number of Professional Hosts',
    yaxis_title='Location',
    yaxis={'categoryorder': 'total ascending'},
    coloraxis_showscale=True 
)


fig1.update_traces(
    textposition='outside',
    texttemplate='%{text:.0f}',
    textfont=dict(color='black')
)



mean_count = location_pro_hosts['NumProfessionalHosts'].mean()
fig1.add_vline(
    x=mean_count,
    line_width=2,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean={mean_count:.2f}",
    annotation_position="top",
    annotation_font=dict(color="red", size=14)
)

fig1.show()



In [ ]:
# 2) Calculate the ratio of professional hosts to total hosts in each location    

location_total_hosts = (
    df.groupby('Location')['Host ID']
    .nunique()
    .reset_index(name='NumTotalHosts')
)

location_pro_hosts = (
    df[df['IsProfessional']]
    .groupby('Location')['Host ID']
    .nunique()
    .reset_index(name='NumProfessionalHosts')
    .sort_values('NumProfessionalHosts', ascending=False)
)



location_stats = pd.merge(
    location_total_hosts, 
    location_pro_hosts,    
    on='Location',
    how='left'  
)
location_stats['NumProfessionalHosts'] = location_stats['NumProfessionalHosts'].fillna(0)

location_stats['ProHostRatio'] = (
    location_stats['NumProfessionalHosts'] / location_stats['NumTotalHosts']
)


top10_merged = pd.merge(
    top10_pro[['Location','NumProfessionalHosts']],  
    location_stats,
    on='Location',
    how='left'
)

top10_merged.drop(columns=['NumProfessionalHosts_x'], inplace=True)
top10_merged.rename(columns={'NumProfessionalHosts_y': 'NumProfessionalHosts'}, inplace=True)




fig2 = px.bar(
    top10_merged,
    x='ProHostRatio',
    y='Location',
    orientation='h',
    title='(2) Same Top 10 (by Professional Host Count) with Ratio',
    color='ProHostRatio',
    color_continuous_scale='Magma',
    text='ProHostRatio' 
)

fig2.update_layout(
    xaxis_title='Pro Host Ratio',
    yaxis_title='Location',
    xaxis_tickformat='.1%',  
    yaxis={'categoryorder': 'array', 'categoryarray': top10_merged['Location'][::-1]},
    height=700,             
    margin=dict(l=200, r=50, t=80, b=50),
    coloraxis_colorbar=dict(
        tickformat='.1%'
    )
)

fig2.update_traces(
    textposition='outside',
    texttemplate='%{text:.1%}',  #
)

global_ratio = (location_stats['NumProfessionalHosts'].sum()) / (location_stats['NumTotalHosts'].sum())
fig2.add_vline(
    x=global_ratio,
    line_width=2,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Avg={global_ratio:.1%}",
    annotation_position="top", 
    annotation_font=dict(color="red", size=14)
)

fig2.show()



In [ ]:
# Filter to locations with >10 total hosts
location_stats_10 = location_stats[location_stats['NumTotalHosts'] > 10].copy()

# Sort by ratio descending
location_stats_10.sort_values('ProHostRatio', ascending=False, inplace=True)

#print("\n=== PART 3: Locations w/ >100 total hosts, sorted by highest ProHostRatio ===")
#print(location_stats_10.head(10))

# Visualize top 10 in that filtered set
top10_ratio = location_stats_10.head(10)

fig3 = px.bar(
    top10_ratio,
    x='ProHostRatio',
    y='Location',
    orientation='h',
    title='(3) Top 10 High-Ratio Locations (TotalHosts > 10)',
    color='ProHostRatio',
    color_continuous_scale='Viridis',
    text='ProHostRatio'  
)

fig3.update_layout(
    xaxis_title='Pro Host Ratio',
    yaxis_title='Location',
    xaxis_tickformat='.1%', 
    yaxis={'categoryorder': 'total ascending'},
    height=700,
    margin=dict(l=200, r=50, t=80, b=50),
    coloraxis_colorbar=dict(
        tickformat='.1%'
    )
)

fig3.update_traces(
    textposition='outside',
    texttemplate='%{text:.1%}',
    textfont=dict(color='black')
)


global_ratio = (location_stats_10['NumProfessionalHosts'].sum()) / (location_stats_10['NumTotalHosts'].sum())
fig3.add_vline(
    x=global_ratio,
    line_width=2,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean={global_ratio:.1%}",
    annotation_position="top",
    annotation_font=dict(color="red", size=14)
)

fig3.show()



In [ ]:
# 1) Calculate how many unique professional hosts in each City
city_pro_hosts = (
    df[df['IsProfessional']]
    .groupby('City')['Host ID']
    .nunique()
    .reset_index(name='NumProfessionalHosts')
    .sort_values('NumProfessionalHosts', ascending=False)
)

top10_city_pro = city_pro_hosts.head(10)


fig_city1 = px.bar(
    top10_city_pro,
    x='NumProfessionalHosts',
    y='City',
    orientation='h',
    title='(1) Top 10 Cities by Count of Professional Hosts',
    color='NumProfessionalHosts',
    color_continuous_scale='Viridis',
    text='NumProfessionalHosts'  
)

fig_city1.update_layout(
    height=700,
    margin=dict(l=200, r=50, t=80, b=50),
    xaxis_title='Number of Professional Hosts',
    yaxis_title='City',
    yaxis={'categoryorder': 'total ascending'},
    coloraxis_showscale=True
)

fig_city1.update_traces(
    textposition='outside',
    texttemplate='%{text:.0f}',
    textfont=dict(color='black')
)

avg_city_count = city_pro_hosts['NumProfessionalHosts'].mean()
fig_city1.add_vline(
    x=avg_city_count,
    line_width=2,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Avg={avg_city_count:.2f}",
    annotation_position="top",
    annotation_font=dict(color="red", size=14)
)

fig_city1.show()



In [ ]:
city_total_hosts = (
    df.groupby('City')['Host ID']
    .nunique()
    .reset_index(name='NumTotalHosts')
)

city_stats = pd.merge(
    city_pro_hosts,
    city_total_hosts,
    on='City',
    how='left'
)

city_stats['ProHostRatio'] = city_stats['NumProfessionalHosts'] / city_stats['NumTotalHosts']

top10_city_merged = pd.merge(
    top10_city_pro[['City','NumProfessionalHosts']],  # same 10 rows
    city_stats,
    on='City',
    how='left'
)

top10_city_merged.drop(columns=['NumProfessionalHosts_x'], inplace=True)
top10_city_merged.rename(columns={'NumProfessionalHosts_y': 'NumProfessionalHosts'}, inplace=True)

fig_city2 = px.bar(
    top10_city_merged,
    x='ProHostRatio',
    y='City',
    orientation='h',
    title='(2) Same Top 10 (by Professional Host Count) with Ratio (City)',
    color='ProHostRatio',
    color_continuous_scale='Magma',
    text='ProHostRatio' 
)

fig_city2.update_layout(
    xaxis_title='Pro Host Ratio',
    yaxis_title='City',
    xaxis_tickformat='.1%',
    yaxis={'categoryorder': 'array', 'categoryarray': top10_city_merged['City'][::-1]},
    height=700,
    margin=dict(l=200, r=50, t=80, b=50),
    coloraxis_colorbar=dict(
        tickformat='.1%'
    )
)

fig_city2.update_traces(
    textposition='outside',
    texttemplate='%{text:.1%}',
    textfont=dict(color='black')
)

avg_city_ratio_all = city_stats['NumProfessionalHosts'].sum() / city_stats['NumTotalHosts'].sum()
fig_city2.add_vline(
    x=avg_city_ratio_all,
    line_width=2,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Avg={avg_city_ratio_all:.1%}",
    annotation_position="top",
    annotation_font=dict(color="red", size=14)
)

fig_city2.show()



In [ ]:
city_stats_10 = city_stats[city_stats['NumTotalHosts'] > 10].copy()

city_stats_10.sort_values('ProHostRatio', ascending=False, inplace=True)

top10_city_ratio_10 = city_stats_10.head(10)

fig_city3 = px.bar(
    top10_city_ratio_10,
    x='ProHostRatio',
    y='City',
    orientation='h',
    title='(3) Top 10 High-Ratio Cities (TotalHosts > 10)',
    color='ProHostRatio',
    color_continuous_scale='Viridis',
    text='ProHostRatio'
)

fig_city3.update_layout(
    xaxis_title='Pro Host Ratio',
    yaxis_title='City',
    xaxis_tickformat='.1%',
    yaxis={'categoryorder': 'total ascending'},
    height=700,
    margin=dict(l=200, r=50, t=80, b=50),
    coloraxis_colorbar=dict(
        tickformat='.1%'
    )
)

fig_city3.update_traces(
    textposition='outside',
    texttemplate='%{text:.1%}',
    textfont=dict(color='black')
)


global_ratio_10 = city_stats_10['NumProfessionalHosts'].sum() / city_stats_10['NumTotalHosts'].sum()
fig_city3.add_vline(
    x=global_ratio_10,
    line_width=2,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Avg={global_ratio_10:.1%}",
    annotation_position="top",
    annotation_font=dict(color="red", size=14)
)

fig_city3.show()
